# Introduction à l'apprentissage automatique: Classification

<br>

### Détection de spam

<br>

Dans ce TP, nous allons entraîner des classifieurs pour décider si un mail est un spam ou non.

<br>

Tout d'abord, quelques indications sur l'utilisation des méthodes d'apprentissage de ̀`scikit-learn`.

Les méthodes d'apprentissage supervisé de `scikit-learn` permettent de définir un objet, doté de différents attributs et méthodes, dont `cross_val_score` (pour calculer un score de validation croisée), `fit` (pour procéder à l'apprentissage), `predict` (pour prédire les classes des éléments d'une base de test), ou `score` pour calculer la proportion d'observations bien classées dans la base de test, sur laquelle on peut comparer la classe prédite à la "vraie" classe.

Ci-dessous, un exemple d'utilisation de la classification au plus proche voisin, dans un scénario où on suppose disposer d'une base d'apprentissage $(X_{train},y_{train})$, et d'une base de test $X_{test}$ pour laquelle on connaît $y_{test}$, de manière à valider l'apprentissage sur la base de test. Si on veut changer de classifieur, il suffit d'utiliser un autre constructeur que `neighbors.KNeighborsClassifier` et de passer les paramètres adéquats.

```python
# (le code suivant ne peut pas être exécuté "tel quel"...)

# classifieur au plus proche voisin (on peut changer le paramètre n_neighbors):
knn = neighbors.KNeighborsClassifier(n_neighbors=1)  

# calcul d'un score moyen de validation croisée "à 5 plis" sur (X_train,y_train)
scores = cross_val_score(knn,X_train,y_train,cv=5)
print("score moyen de validation croisée: %0.3f (+/- %0.3f)" % (scores.mean(),2*scores.std()))

# la prédiction d'une nouvelle observation consistera à chercher le p.p.v. dans X_train,
# et à associer la classe de ce p.p.v., donnée par y_train:
knn.fit(X_train,y_train)  
# Remarque: il n'y a pas d'apprentissage à proprement parler pour les p.p.v.,
# il s'agit juste de préciser la base dans laquelle seront cherchés les plus proches voisins

# on stocke dans y_pred les classes prédites sur un ensemble de test X_test:
y_pred = knn.predict(X_test)  

# calcul d'un score lorsqu'on connaît les vraies classes des observations de X_test:
# (proportion d'observations pour lesquelles y_test==y_pred)
score = knn.score(X_test,y_test)
```

## Préliminaires

<br>

Commençons par charger les bibliothèques utiles.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import neighbors, linear_model, naive_bayes, metrics
%matplotlib inline

Ensuite, on charge les données: récupérez au préalable le fichier `spambase.data` disponible sur le [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/94/spambase) (Cliquez sur "Download" pour télécharger les fichiers). La description complète de la base est dans le fichier `spambase.name`, à ouvrir avec un éditeur de texte.

<br>

La cellule suivante charge les données. On forme une base d'entraînement avec 80% des données (choix aléatoire), et on garde 20% des données pour faire une base de test. Dans la cellule suivante, on fixe la graîne du générateur aléatoire (`random_state=42`, la valeur est arbitraire) de manière à ce que l'on ait tous les mêmes résultats afin de faciliter la comparaison.


In [ ]:
data = np.loadtxt('spambase.data', delimiter=',')
X_train, X_test, y_train, y_test = train_test_split(data[:,:-1], data[:,-1], test_size=0.2, random_state=42)
# pour vérifier que les données sont bien chargées:
print(data)
print(data.shape)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_train)
print(y_train)

[[0.000e+00 6.400e-01 6.400e-01 ... 6.100e+01 2.780e+02 1.000e+00]
 [2.100e-01 2.800e-01 5.000e-01 ... 1.010e+02 1.028e+03 1.000e+00]
 [6.000e-02 0.000e+00 7.100e-01 ... 4.850e+02 2.259e+03 1.000e+00]
 ...
 [3.000e-01 0.000e+00 3.000e-01 ... 6.000e+00 1.180e+02 0.000e+00]
 [9.600e-01 0.000e+00 0.000e+00 ... 5.000e+00 7.800e+01 0.000e+00]
 [0.000e+00 0.000e+00 6.500e-01 ... 5.000e+00 4.000e+01 0.000e+00]]
(4601, 58)
(3680, 57)
(3680,)
(921, 57)
(921,)
[[9.000e-02 0.000e+00 9.000e-02 ... 6.813e+00 4.940e+02 1.458e+03]
 [0.000e+00 0.000e+00 0.000e+00 ... 1.800e+00 8.000e+00 4.500e+01]
 [0.000e+00 0.000e+00 2.430e+00 ... 2.319e+00 1.200e+01 1.670e+02]
 ...
 [0.000e+00 0.000e+00 0.000e+00 ... 2.333e+00 1.000e+01 4.900e+01]
 [0.000e+00 2.300e-01 0.000e+00 ... 1.616e+00 1.300e+01 1.730e+02]
 [1.000e-01 0.000e+00 4.100e-01 ... 1.915e+00 2.900e+01 3.390e+02]]
[1. 0. 0. ... 0. 0. 1.]


__Question 1__. A partir de la description de la base de données, justifiez la manière employée pour charger les données en `X` (observations) et `y` (labels). Quelles sont les caractéristiques des observations, les labels, et quel est le rapport avec le problème initial?

<font color=red>
Votre réponse: <br>
Cette base de données contient des caractéristiques extraites d'e-mails, notamment la fréquence d'apparition de certains mots, de certains caractères, ainsi que des informations sur l'utilisation des majuscules.<br><br>
L'objectif du problème initial est de classer les e-mails en tant que spam ou non-spam en fonction de ces caractéristiques extraites. Par conséquent, les observations (X) correspondent aux caractéristiques extraites des e-mails, et les labels (y) correspondent à la classification en spam (1) ou non-spam (0), c'est de la  classification binaire. <br>
Il y a 48 attributs continus de word_freq_WORD, les 6 attributs continus de char_freq_CHAR, ainsi que les trois attributs continus de capital_run_length_average, capital_run_length_longest, et capital_run_length_total comme variables explicatives (X), et l'attribut nominal spam comme la variable cible (y).
    
</font>

## Classification aux plus proches voisins

<br>

Mettez en oeuvre les classifications au plus proche voisin et aux 5 plus proches voisins. Vous calculerez le score moyen de validation croisée à 5 plis sur la base d'apprentissage ainsi que le score obtenu sur la base de test. Vous vous inspirerez du code détaillé en introduction.

__Question 2__. Quelle est la métrique utilisée pour déterminer les plus proches voisins? Quel est ce "score" calculé exactement? Quel lien entre score de validation croisée et score sur la base de test?

In [ ]:
# votre code ici pour 1-ppv:
knn = neighbors.KNeighborsClassifier(n_neighbors=1)

# calcul d'un score moyen de validation croisée "à 5 plis" sur (X_train,y_train)
scores = cross_val_score(knn,X_train,y_train,cv=5)
print("score moyen de validation croisée: %0.3f (+/- %0.3f)" % (scores.mean(),2*scores.std()))

# la prédiction d'une nouvelle observation consistera à chercher le p.p.v. dans X_train,
# et à associer la classe de ce p.p.v., donnée par y_train:
knn.fit(X_train,y_train)
# Remarque: il n'y a pas d'apprentissage à proprement parler pour les p.p.v.,
# il s'agit juste de préciser la base dans laquelle seront cherchés les plus proches voisins

# on stocke dans y_pred les classes prédites sur un ensemble de test X_test:
y_pred = knn.predict(X_test)

# calcul d'un score lorsqu'on connaît les vraies classes des observations de X_test:
# (proportion d'observations pour lesquelles y_test==y_pred)
score = knn.score(X_test,y_test)
print("score moyen de test : %0.3f (+/- %0.3f)" % (score.mean(),2*score.std()))

score moyen de validation croisée: 0.803 (+/- 0.016)
score moyen de test : 0.808 (+/- 0.000)


In [ ]:
y_pred = knn.predict(X_test)
y_pred[:10]

array([0., 1., 0., 0., 0., 0., 0., 1., 0., 0.])

In [ ]:
y_test[:10]

array([0., 0., 0., 1., 0., 1., 0., 0., 0., 0.])

In [ ]:
# votre code ici pour 5-ppv:
knn = neighbors.KNeighborsClassifier(n_neighbors=5)

# calcul d'un score moyen de validation croisée "à 5 plis" sur (X_train,y_train)
scores = cross_val_score(knn,X_train,y_train,cv=5)
print("score moyen de validation croisée: %0.3f (+/- %0.3f)" % (scores.mean(),2*scores.std()))

# la prédiction d'une nouvelle observation consistera à chercher le p.p.v. dans X_train,
# et à associer la classe de ce p.p.v., donnée par y_train:
knn.fit(X_train,y_train)
# Remarque: il n'y a pas d'apprentissage à proprement parler pour les p.p.v.,
# il s'agit juste de préciser la base dans laquelle seront cherchés les plus proches voisins

# on stocke dans y_pred les classes prédites sur un ensemble de test X_test:
y_pred_kn1 = knn.predict(X_test)

# calcul d'un score lorsqu'on connaît les vraies classes des observations de X_test:
# (proportion d'observations pour lesquelles y_test==y_pred)
score = knn.score(X_test,y_test)
print("score moyen de test : %0.3f (+/- %0.3f)" % (score.mean(),2*score.std()))

score moyen de validation croisée: 0.791 (+/- 0.023)
score moyen de test : 0.790 (+/- 0.000)


<font color=red>
Votre réponse:<br>
La métrique utilisée pour déterminer les plus proches voisins est la distance euclidienn (métrique par défaut pour le classificateur k des plus proches voisins (k-NN) de scikit-learn). La distance euclidienne mesure la distance spatiale entre deux points.<br>
Le "score" calculé exactement dans le code est la proportion d'observations correctement classées des y_test, c'est la précision de la classification des valeurs.<br>
Le score de validation croisée est calculé en utilisant une validation croisée à 5 plis sur la base d'apprentissage. Il divise la base d'apprentissage en 5 sous-ensembles (plis), utilise 4 d'entre eux pour l'apprentissage et le dernier pour la validation. Celui-ci utilise les x_train seulement. Alors qu'à contrario, le score sur la base de test est calculé en utilisant le modèle entraîné sur la base d'apprentissage pour prédire les classes de la base de test.<br>
Ici les scores sont similaires ce qui peut suggérer que le modèle d'entrainement est plutot bon.
</font>

__Question 4__. Pré-traitez les données par standardisation, comme expliqué ici sur [la documentation scikit-learn](https://scikit-learn.org/stable/modules/preprocessing.html) (utilisez le `StandardScaler` comme indiqué dans la 3ème cellule, de manière à utiliser la même normalisation sur la base d'apprentissage et sur la base de test, c'est important), puis recalculez les scores des deux classifieurs précédents.

In [ ]:
# votre code:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(X_train)

X_scaled = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)


knn = neighbors.KNeighborsClassifier(n_neighbors=5)

# calcul d'un score moyen de validation croisée "à 5 plis" sur (X_train,y_train)
scores = cross_val_score(knn,X_scaled,y_train,cv=5)
print("score moyen de validation croisée: %0.3f (+/- %0.3f)" % (scores.mean(),2*scores.std()))

# la prédiction d'une nouvelle observation consistera à chercher le p.p.v. dans X_train,
# et à associer la classe de ce p.p.v., donnée par y_train:
knn.fit(X_scaled,y_train)
# Remarque: il n'y a pas d'apprentissage à proprement parler pour les p.p.v.,
# il s'agit juste de préciser la base dans laquelle seront cherchés les plus proches voisins

# on stocke dans y_pred les classes prédites sur un ensemble de test X_test:
y_pred_kn5 = knn.predict(X_scaled_test)

# calcul d'un score lorsqu'on connaît les vraies classes des observations de X_test:
# (proportion d'observations pour lesquelles y_test==y_pred)
score = knn.score(X_scaled_test,y_test)
print("score moyen de test : %0.3f (+/- %0.3f)" % (score.mean(),2*score.std()))

score moyen de validation croisée: 0.906 (+/- 0.013)
score moyen de test : 0.894 (+/- 0.000)


In [ ]:
X_train[:2,:]

array([[9.000e-02, 0.000e+00, 9.000e-02, 0.000e+00, 3.900e-01, 9.000e-02,
        9.000e-02, 0.000e+00, 1.900e-01, 2.900e-01, 3.900e-01, 4.800e-01,
        0.000e+00, 5.800e-01, 0.000e+00, 8.700e-01, 1.900e-01, 0.000e+00,
        1.660e+00, 4.100e+00, 1.660e+00, 0.000e+00, 3.900e-01, 1.900e-01,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 1.390e-01, 0.000e+00, 3.100e-01, 1.550e-01, 0.000e+00,
        6.813e+00, 4.940e+02, 1.458e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.580e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        7.900e-01, 0.000e+00, 7.900e-01, 0.000e+00, 0.000e+00, 0.000e+

In [ ]:
X_scaled[:2,:]

array([[-3.84417967e-02, -1.62823436e-01, -3.73283587e-01,
        -4.43387016e-02,  1.25329068e-01, -1.75557108e-02,
        -4.94618697e-02, -2.58223628e-01,  3.74143676e-01,
         7.84018383e-02,  1.64169725e+00, -7.44333103e-02,
        -3.27398098e-01,  1.51199468e+00, -1.90055678e-01,
         7.05973261e-01,  1.15944336e-01, -3.45099898e-01,
         6.08468580e-03,  7.83188746e+00,  7.10708263e-01,
        -1.16300745e-01,  8.23756701e-01,  2.01038747e-01,
        -3.26710260e-01, -3.00878899e-01, -2.30922222e-01,
        -2.31071959e-01, -1.72682655e-01, -2.24879107e-01,
        -1.74395864e-01, -1.43223289e-01, -1.73866411e-01,
        -1.45765474e-01, -1.93393004e-01, -2.42335411e-01,
        -3.30036623e-01, -5.64810168e-02, -1.78189932e-01,
        -1.86445138e-01, -1.21635838e-01, -1.74633824e-01,
        -2.03750279e-01, -1.27828730e-01, -2.99465798e-01,
        -2.11088191e-01, -7.04194454e-02, -1.17147007e-01,
        -1.57562225e-01, -8.70397336e-03, -1.73306666e-0

<font color=red>
    Votre réponse:<br>
    
Dans ce code, nous effectuons le prétraitement des données en utilisant la standardisation à l'aide de StandardScaler de scikit-learn. Ensuite, nous recalculons les scores des classifieurs k-NN (k plus proches voisins) sur la base d'apprentissage et de test après ce prétraitement.<br><br>
La standardisation des données signifie que nous centrons les données autour de zéro (moyenne de 0) et les mettons à l'échelle pour avoir une variance de 1.<br><br>
Le score moyen de validation croisée est élevé (environ 0.906), ce qui indique que le modèle k-NN fonctionne bien sur la base d'apprentissage après la standardisation des données.
Le score moyen de test est également élevé (environ 0.894), ce qui suggère que le modèle généralise bien sur de nouvelles données (base de test).<br>
Pour rappel les scores sans pré-traitement des données était nettement plus faible : <br><br>
score moyen de validation croisée: 0.791 (+/- 0.023)
score moyen de test : 0.790 (+/- 0.000)
</font>

## Classifieur naïf de Bayes gaussien et régression logistique

<br>


__Question 5__. Pourquoi le classifieur naïf de Bayes gaussien ne nécessite-t-il pas de standardisation préalable des données ? (vous pouvez vérifier que la normalisation joue tout de même un faible rôle: elle a sans doute une influence sur le comportement de l'algorithme d'estimation des paramètres).


Mettez en oeuvre le classifieur naïf de Bayes gaussien (lisez le début de la [documentation](https://scikit-learn.org/stable/modules/naive_bayes.html) où vous retrouverez le contenu du cours, puis la syntaxe de `GaussianNB` [ici](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html)), ainsi que le classifieur de la régression logistique ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)).

Pour ce dernier classifieur, passez l'option `max_iter=2000` si vous avez un avertissement concernant la convergence de l'optimisation, de la manière suivante:
`LR = linear_model.LogisticRegression(max_iter=2000)`

In [ ]:
# votre code pour le classifieur naïf de Bayes gaussien (sur les données originales):
from sklearn import naive_bayes

# votre code pour le classifieur naif Gaussien:
naive = naive_bayes.GaussianNB()
naive.fit(X_train, y_train)
%time y_pred_naive = naive.predict(X_test)
print('naive score: %f' % metrics.accuracy_score(y_test, y_pred_naive))


CPU times: user 2.2 ms, sys: 11 µs, total: 2.21 ms
Wall time: 17.8 ms
naive score: 0.820847


In [ ]:
# votre code pour la régression logistique (sur les données standardisées):
from sklearn import linear_model

linear = linear_model.LogisticRegression(max_iter=2000)
linear.fit(X_train, y_train)
%time y_pred_linear = linear.predict(X_test)
print('linear score: %f' % metrics.accuracy_score(y_test, y_pred_linear))


CPU times: user 364 µs, sys: 789 µs, total: 1.15 ms
Wall time: 627 µs
linear score: 0.921824


<font color=red>
Votre réponse:<br>
Dans la classification par naive gaussienne, la formule qui nous permet de calculer "y" comporte déjà un pré traitement des données car on fait (les valeurs d'entrées - la moyenne des y)² / l'écart type au carré.<br><br>
Ceci permet de pré-traiter les données.

    
</font>

## Analyse des résultats

<br>

On dispose des matrices de confusion, décrites [ici](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html), et des rapports de classification, décrits [là](https://scikit-learn.orgz/stable/modules/generated/sklearn.metrics.classification_report.html).

__Question 6__. Affichez ces matrices et rapports pour les quatre classifieurs testés.

In [ ]:
# votre code ici:

#librairie
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
#matrice de confusion
matrice_linear = confusion_matrix(y_test, y_pred_linear)
matrice_naive = confusion_matrix(y_test, y_pred_naive)
matrice_kn1 = confusion_matrix(y_test, y_pred_kn1)
matrice_kn5 = confusion_matrix(y_test, y_pred_kn5)
print("matrice linear : \n", matrice_linear,"\nmatrice naive \n",matrice_naive ,"\nmatrice kn1 \n", matrice_kn1,"\nmatrice kn5 \n",matrice_kn5)

#rapports de classifications
class_linear = classification_report(y_test, y_pred_linear)
class_naive = classification_report(y_test, y_pred_naive)
class_kn1 = classification_report(y_test, y_pred_kn1)
class_kn5 = classification_report(y_test, y_pred_kn5)
print("\nclassification linear : \n", class_linear,"\nclassification naive \n",class_naive ,"\nclassification kn1 \n", class_kn1,"\nclassification kn2 \n",class_kn5)

matrice linear : 
 [[505  26]
 [ 46 344]] 
matrice naive 
 [[387 144]
 [ 21 369]] 
matrice kn1 
 [[450  81]
 [112 278]] 
matrice kn5 
 [[494  37]
 [ 61 329]]

classification linear : 
               precision    recall  f1-score   support

         0.0       0.92      0.95      0.93       531
         1.0       0.93      0.88      0.91       390

    accuracy                           0.92       921
   macro avg       0.92      0.92      0.92       921
weighted avg       0.92      0.92      0.92       921
 
classification naive 
               precision    recall  f1-score   support

         0.0       0.95      0.73      0.82       531
         1.0       0.72      0.95      0.82       390

    accuracy                           0.82       921
   macro avg       0.83      0.84      0.82       921
weighted avg       0.85      0.82      0.82       921
 
classification kn1 
               precision    recall  f1-score   support

         0.0       0.80      0.85      0.82       531
      

__Question 7__. Ici, par quoi pourraient s'expliquer les performances modestes du classifieur naïf de Bayes ?
A ce stade, quel classifieur préfére-t-on et pourquoi? Dans une application de détection de spams, cherche-t-on réellement à minimiser le taux d'erreur global?

<font color=red>
Réponse: <br>
Le classifieur le plus performant est sans aucun doute la régression logistique, on peut l'identifier grâce à son nombre élever de vrai positif et vrai négatif, son nombre faible de faux négatif et faux positif, mais aussi grâce à ses scores dans le rapport de classifications.
<BR>
Mais le classifieur naive est intéressant car il y a moins de faux positif, donc moins de mails non spam qui iront dans les spams.
</font>

Le classifieur bayésien naïf gaussien et le classifieur de la régression logistique s'appuient tous deux sur la règle du maximum a posteriori. Ils permettent d'estimer la probabilité a posteriori $p(C_1|x)$ et détectent un spam lorsque $p(C_1|x)>1/2$, où $C_1$ désigne la classe "spam" et $x$ est une observation. Les deux classifieurs mettent en oeuvre le classifieur de Bayes, qui minimise le risque moyen de prédiction (le taux d'erreur).  Le taux d'erreur "compte" de la même manière les erreurs sur les deux classes.

Si on préfère réduire le taux de faux positif de la méthode (proportion de mails détectés à tort comme "spam"), on peut relever le seuil de probabilité.

Les classifieurs `LogisticRegression` et `GaussianNB` possèdent tous deux une méthode `predict_proba` qui, pour un tableau d'observations, fournit la probabilité a posteriori de chaque classe, comme l'affiche la cellule suivante. On remarque que pour chaque observation $x$, $p(C_0|x)+p(C_1|x)=1$.  (attention, la documentation n'est pas très claire, `predict_proba` fournit bien la probabilité a posteriori, et pas la vraisemblance $p(x|C_k)$)

Remarquons qu'aucune probabilité n'est fournie par la classification aux plus proches voisins.

In [ ]:
print("probabilités a posteriori pour naive:")
print(naive.predict_proba(X_test))
print("\nprobabilités a posteriori pour linear:")
print(linear.predict_proba(X_scaled_test))

probabilités a posteriori pour GNB:
[[5.19907033e-13 1.00000000e+00]
 [1.00000000e+00 3.06819165e-10]
 [1.00000000e+00 6.84756259e-16]
 ...
 [1.00000000e+00 0.00000000e+00]
 [9.99218390e-01 7.81609538e-04]
 [1.00000000e+00 0.00000000e+00]]

probabilités a posteriori pour LR:
[[1.60047981e-06 9.99998400e-01]
 [9.94232826e-01 5.76717356e-03]
 [9.34978227e-01 6.50217731e-02]
 ...
 [9.99976336e-01 2.36644591e-05]
 [9.75137668e-01 2.48623318e-02]
 [3.39631313e-03 9.96603687e-01]]


In [ ]:
p=0.85  # constatez que p=0.5 fournit les mêmes résultats pour y_pred_lr et y_pred_LRb
y_pred_linear = (linear.predict_proba(X_scaled_test)[:,1] >= p).astype(int)
#print(y_pred_LRb)   # pour visualiser les classes prédites
#print(y_pred_lr)
score_linear = 1-np.mean(np.abs(y_test-y_pred_linear))  # calcul du taux de reconnaissance

print("\nRégression logistique à seuil:")
print('score sur la base de test: %2f' %score_linear)
print(metrics.classification_report(y_test,y_pred_linear))
print("matrice de confusion:")
print(metrics.confusion_matrix(y_test,y_pred_linear))




Régression logistique à seuil:
score sur la base de test: 0.883822
              precision    recall  f1-score   support

         0.0       0.85      0.97      0.91       531
         1.0       0.95      0.77      0.85       390

    accuracy                           0.88       921
   macro avg       0.90      0.87      0.88       921
weighted avg       0.89      0.88      0.88       921

matrice de confusion:
[[514  17]
 [ 90 300]]


__Question 8__. Quelle valeur du seuil de probabilité $p$ faut-il choisir pour assurer un rappel de la classe "non spam" d'au moins 0.98?
Que penser de cet algorithme de détection de spam?

<font color=red>
    
Réponse:
    
</font>


In [ ]:
p = 0.85